1.1 The Clique Functions in the Ising Model

In [7]:
#Computer Exercise 1
import numpy as np
def G(row_s: np.ndarray, Temp: float):
    return np.exp(np.sum(row_s[:-1] * row_s[1:]) / Temp)
 


In [12]:
#Computer Exercise 2

def F(row_s: np.ndarray, row_t: np.ndarray, Temp: float):
    return np.exp(np.sum(row_s * row_t) / Temp)


1.2 Brute Force on Small Lattices
Some of the computer exercises in this section require you to use an absurd
number (from a programmer’s perspective) of nested loops. Thus, you can
(but do not have to) exploit Python’s itertools module to make this a bit more
elegant.
compute ZTemp (for three different values of Temp where Temp ∈ {1, 1.5, 2})
using brute force (use 4 nested For loops, one for each of the xs’s, the looping
is done over the values that xs can take: {−1, 1}). To help you debug: For
Temp = 1, your result should be ZTemp = 121.23 . . ..

In [19]:
from itertools import product
rows = np.asarray(list(product([-1, 1], repeat=2)))
print(rows)


[[-1 -1]
 [-1  1]
 [ 1 -1]
 [ 1  1]]


In [62]:
#Computer Exercise 3
from itertools import product

def compute_ZTemp1(Temp: float):
    Z = 0
    vals = [-1, 1]

    for x11 in vals:
        for x12 in vals:
            for x21 in vals:
                for x22 in vals:
                    Z += np.exp((x11*x21+ x12*x22 + x11*x12 + x21*x22)/Temp)

    return Z

temp = [1,1.5,2]
for i in temp:
    z = compute_ZTemp1(i)
    print(z)
    

121.23293134406595
40.922799092745386
27.048782764334526


In [28]:
#Computer Exercise 4

import numpy as np

import numpy as np

def compute_ZTemp2(Temp: float):
    Z = 0
    vals = [-1, 1]

    for x11 in vals:
        for x12 in vals:
            for x13 in vals:
                for x21 in vals:
                    for x22 in vals:
                        for x23 in vals:
                            for x31 in vals:
                                for x32 in vals:
                                    for x33 in vals:
                                        energy = x11*x21 + x12*x22 + x13*x23 + x21*x31 + x22*x32 + x23*x33 + x11*x12 + x12*x13 + x21*x22 + x22*x23 + x31*x32 + x32*x33
                                        Z += np.exp(energy/Temp)

    return Z




temp = [1,1.5,2]
for i in temp:
    z = compute_ZTemp2(i)
    print(z)

365645.74913577037
10565.421983514265
2674.518123060087


In [35]:
# computer exercise 5

def y2row(y, width=2):
    """
    y: an integer in (0,...,(2**width)-1)
    """
    if not 0 <= y <= (2 ** width) - 1:
        raise ValueError(y)
    my_str=np.binary_repr(y,width=width)
    # my_list = map(int,my_str) # Python 2
    my_list = list(map(int,my_str)) # Python 3
    my_array = np.asarray(my_list)
    my_array[my_array==0]=-1
    row = my_array
    return row

def compute_ZTemp3(Temp: float):
    Z = 0
    ys = [0,1,2,3]
    rows = []
    for y in ys:
        rows.append(y2row(y, width=2))                       # convert to row vectors  

    rows = np.asarray(rows)                                  # convert to numpy array
    
    for y1 in rows:
        for y2 in rows:
            Z += G(y1, Temp) * G(y2, Temp) * F(y1, y2, Temp) # required logic
            
    return Z

In [39]:
temp = [1,1.5,2]
for i in temp:
    z = compute_ZTemp3(i)
    print(z)

121.23293134406595
40.922799092745386
27.048782764334526


In [41]:
# computer exercise 6
def compute_ZTemp4(Temp: float):
    Z = 0
    ys = [0,1,2,3,4,5,6,7]
    rows = []
    for y in ys:
        rows.append(y2row(y, width=3))                       # convert to row vectors  

    rows = np.asarray(rows)                                  # convert to numpy array
    
    for y1 in rows:
        for y2 in rows:
            for y3 in rows:
                Z += G(y1, Temp) * G(y2, Temp) * G(y3, Temp) * F(y1, y2, Temp) * F(y2, y3, Temp)
    return Z 

In [42]:
temp = [1,1.5,2]
for i in temp:
    z = compute_ZTemp4(i)
    print(z)

365645.7491357704
10565.421983514265
2674.518123060087


1.3 Dynamic Programming on an 8×8 Lattice

In [80]:
#computer exercise 7

def get_Gs(size: int, Temp: float):
    """
    size: size of the lattice
    Temp: temperature
    """
    G_MAT = np.asarray([G(y2row(i, width=size), Temp) for i in range(2 ** size)])
    return G_MAT

def get_Fs(size: int, Temp: float):
    """
    size: size of the lattice
    Temp: temperature
    """
    F_MAT = np.asarray([[F(y2row(i, width=size), y2row(j, width=size), Temp) for j in range(2 ** size)] for i in
                            range(2 ** size)])
    return F_MAT


def get_Ts(size,temp):
    T = np.zeros((size - 1, 2 ** size))
    G_MAT = get_Gs(size, temp)
    F_MAT = get_Fs(size, temp)

    for ti in range(size - 1):
        for yi in range(2 ** size):
            if ti == 0:
                T[ti, yi] = np.sum([G_MAT[i] * F_MAT[i, yi] for i in range(2 ** size)]) # equation - (18)
            else:
                T[ti, yi] = np.sum(
                    [T[ti - 1, i] * G_MAT[i] * F_MAT[i, yi] for i in range(2 ** size)]) # equation - (19)
            
    ZTemp = np.sum(T[-1, :] * G_MAT) # equation (20)
        
    return T, ZTemp

def get_Ps(size,temp): # doesnt work correctly
    T, ZTemp = get_Ts(size,temp)
    G_MAT = get_Gs(size, temp)
    F_MAT = get_Fs(size, temp)
    P = []
    #loop from size-1 to 0
    for i in range(size-1,-1,-1):
        if i == size-1:
            P.append([T[size-2,yi]*G_MAT[yi]/ZTemp for yi in range(2**size)]) #equation (21)
        elif i == 0:
            P.append([G_MAT[yi]*F_MAT[yi,yj]/T[0,yj] for yi,yj in product(range(2**size),range(2**size))]) #equation (23)

        else:
            P.append([T[i-1,yi]*G_MAT[yi]*F_MAT[yi,yj]/T[i,yj] for yi,yj in product(range(2**size),range(2**size))]) #equation (22)

    return P
            


# def calc_probs(size, ztemp):
#     T,ztemp = get_Ts(size, ztemp)
#     G_MAT = get_Gs(size, ztemp)
#     F_MAT = get_Fs(size, ztemp)
#     P = []
#     for row in range(size - 1, -1, -1):
#         if row == size - 1:
#             P.insert(0, [(T[row - 1, i] * G_MAT[i]) / ztemp for i in range(2 ** size)])
#         elif row == 0:
#             P.insert(0, [[(G_MAT[i] * F_MAT[i, j]) / T[row, j] for j in range(2 ** size)] for i in range(2 ** size)])
#         else:
#             P.insert(0, [[(T[row - 1, i] * G_MAT[i] * F_MAT[i, j]) / T[row, j] for j in range(2 ** size)] for i in
#                       range(2 ** size)])
#     return P


# print(get_Ps(2,1))
# print("*********")
# print(calc_probs(2,1))

[[0.4751031703562245, 0.024896829643775445, 0.024896829643775445, 0.4751031703562245], [0.9479149938275155, 0.3313106114932943, 0.3313106114932943, 0.017361668724161464, 0.017361668724161464, 0.3313106114932944, 0.006068165520116885, 0.017361668724161464, 0.017361668724161464, 0.006068165520116885, 0.3313106114932944, 0.017361668724161464, 0.017361668724161464, 0.3313106114932943, 0.3313106114932943, 0.9479149938275155]]
*********
[[[0.048376326919623604, 0.12289187730802734, 0.12289187730802734, 0.04680622680316243], [0.04680622680316243, 0.12289187730802734, 0.11890330349187024, 0.04680622680316243], [0.04680622680316243, 0.11890330349187024, 0.12289187730802734, 0.04680622680316243], [0.04680622680316243, 0.12289187730802734, 0.12289187730802734, 0.048376326919623604]], [0.1762283443229034, 0.0671206595115202, 0.0671206595115202, 0.1762283443229034]]
